**Vector Store Creation**

Sources:
[Article](https://towardsdatascience.com/running-llama-2-on-cpu-inference-for-document-q-a-3d636037a3d8)

Embedding Models
- https://huggingface.co/sentence-transformers/all-mpnet-base-v2
- https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
- https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2

In [1]:
%pip install langchain

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -U sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
# implement pdf reader

/Users/josi/Library/Python/3.8/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Document loader
loader = TextLoader("./index.txt")
documents = loader.load()

with open("./index.txt") as f:
    document = f.read()

In [6]:
# normal text splitter
raw_documents = TextLoader("./index.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(raw_documents)

In [7]:
# Recursive text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len,
    add_start_index = True,
)

In [8]:
# Embedding model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embedding = model.encode(document)
embedding

.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 95.7kB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 125kB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 5.63MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 93.4kB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 19.6MB/s]
pytorch_model.bin: 100%|██████████| 438M/438M [00:34<00:00, 12.7MB/s] 
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 18.2kB/s]
special_tokens_map.json: 100%|██████████| 239/239 [00:00<00:00, 68.8kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 12.7MB/s]
tokenizer_config.json: 100%|██████████| 363/363 [00:00<00:00, 130kB/s]
train_script.py: 100%|██████████| 13.1k/13.1k [00:00<00:00, 4.06MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 14.8MB/s]
modules.json: 100%|██████████| 349/349 [00:00<00:00, 119kB/s]


array([ 4.83290851e-02, -2.01610904e-02, -5.30854380e-03,  5.28399721e-02,
       -1.64846554e-02,  7.25408690e-03,  2.25059967e-02, -2.40077497e-03,
       -8.50354601e-03, -4.31195945e-02, -3.02975960e-02, -3.46438517e-03,
       -6.80337474e-03, -9.49671026e-03,  9.43800062e-03, -6.88497797e-02,
       -1.15029495e-02,  3.47665027e-02, -2.86825672e-02,  1.21779805e-02,
       -3.99886072e-02,  1.22032473e-02,  1.25161074e-02,  3.29638571e-02,
        1.01778377e-02,  2.22062543e-02,  1.28913438e-02, -4.25408334e-02,
        3.37343253e-02, -3.13008949e-02,  3.05749886e-02, -1.59124900e-02,
       -1.86159462e-02, -4.24512252e-02,  2.55800433e-06,  1.28191919e-03,
        4.13662232e-02,  2.86241225e-03, -4.05562902e-03, -2.74212472e-02,
       -7.99735170e-03, -2.12476589e-02, -2.52030455e-02, -3.24683599e-02,
        5.75710554e-03, -3.43444534e-02, -7.55964126e-03,  6.00730889e-02,
        1.96933895e-02, -2.76555512e-02,  2.11346000e-02,  1.25717791e-02,
        1.30772758e-02,  

In [9]:
texts = text_splitter.create_documents([document])
print(texts[0])
print(texts[1])

page_content='In Südafrika findet man eine beeindruckende Vielfalt an Tierarten, die von den weitläufigen Savannen' metadata={'start_index': 0}
page_content='Savannen bis hin zu den dichten' metadata={'start_index': 92}


In [10]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = FAISS.from_documents(docs, embedding_function)

query = "Was für Vogelarten gibt es?"
docs = db.similarity_search(query)

print(docs[0].page_content)

.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 140kB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 56.6kB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 3.23MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 37.6kB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 11.1MB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:07<00:00, 13.0MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 18.4kB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 40.5kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 1.53MB/s]
tokenizer_config.json: 100%|██████████| 350/350 [00:00<00:00, 123kB/s]
train_script.py: 100%|██████████| 13.2k/13.2k [00:00<00:00, 4.34MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 16.0MB/s]
modules.json: 100%|██████████| 349/349 [00:00<00:00, 128kB/s]


Neben diesen bekannten Arten gibt es auch eine Vielzahl an Antilopenarten, wie die schnellen Impalas oder die majestätischen 
Kudus, die sich durch ihre eleganten Hörner auszeichnen. Die Leoparden und Geparden, zwei weitere beeindruckende Raubkatzen, 
sind für ihre Schnelligkeit und Geschicklichkeit bei der Jagd bekannt. In den Flüssen und Feuchtgebieten Südafrikas findet man 
Nilpferde und Krokodile, die trotz ihrer Unterschiedlichkeit beide gefürchtete Bewohner ihres Lebensraums sind.

Die Vielfalt der Vogelwelt in Südafrika ist ebenfalls bemerkenswert, mit Arten wie dem farbenprächtigen Bienenfresser und dem 
majestätischen Afrikanischen Fischadler. Nicht zu vergessen sind die zahlreichen kleineren Tiere wie Warzenschweine, 
Erdmännchen und verschiedene Arten von Affen, die alle zum einzigartigen Ökosystem Südafrikas beitragen.


In [48]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

model = SentenceTransformer('paraphrase-MiniLM-L12-v2')
embedding = model.encode(document)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L12-v2")

db = FAISS.from_documents(docs, embedding_function)

query = "Was für Vogelarten gibt es?"
docs = db.similarity_search(query)

print(docs[0].page_content)

Neben diesen bekannten Arten gibt es auch eine Vielzahl an Antilopenarten, wie die schnellen Impalas oder die majestätischen 
Kudus, die sich durch ihre eleganten Hörner auszeichnen. Die Leoparden und Geparden, zwei weitere beeindruckende Raubkatzen, 
sind für ihre Schnelligkeit und Geschicklichkeit bei der Jagd bekannt. In den Flüssen und Feuchtgebieten Südafrikas findet man 
Nilpferde und Krokodile, die trotz ihrer Unterschiedlichkeit beide gefürchtete Bewohner ihres Lebensraums sind.

Die Vielfalt der Vogelwelt in Südafrika ist ebenfalls bemerkenswert, mit Arten wie dem farbenprächtigen Bienenfresser und dem 
majestätischen Afrikanischen Fischadler. Nicht zu vergessen sind die zahlreichen kleineren Tiere wie Warzenschweine, 
Erdmännchen und verschiedene Arten von Affen, die alle zum einzigartigen Ökosystem Südafrikas beitragen.


In [49]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embedding = model.encode(document)

embedding_function = SentenceTransformerEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

db = FAISS.from_documents(docs, embedding_function)

query = "Was für Vogelarten gibt es?"
docs = db.similarity_search(query)

print(docs[0].page_content)

Neben diesen bekannten Arten gibt es auch eine Vielzahl an Antilopenarten, wie die schnellen Impalas oder die majestätischen 
Kudus, die sich durch ihre eleganten Hörner auszeichnen. Die Leoparden und Geparden, zwei weitere beeindruckende Raubkatzen, 
sind für ihre Schnelligkeit und Geschicklichkeit bei der Jagd bekannt. In den Flüssen und Feuchtgebieten Südafrikas findet man 
Nilpferde und Krokodile, die trotz ihrer Unterschiedlichkeit beide gefürchtete Bewohner ihres Lebensraums sind.

Die Vielfalt der Vogelwelt in Südafrika ist ebenfalls bemerkenswert, mit Arten wie dem farbenprächtigen Bienenfresser und dem 
majestätischen Afrikanischen Fischadler. Nicht zu vergessen sind die zahlreichen kleineren Tiere wie Warzenschweine, 
Erdmännchen und verschiedene Arten von Affen, die alle zum einzigartigen Ökosystem Südafrikas beitragen.


In [ ]:
# Vector Store

In [ ]:
# Retrievers

# ChromaDB, Faiss oder Pinecone: recherchieren
# RecursiveCharacterTextSplitter (empfohlen) vs Normal splitter (eher nicht nutzen) 

In [ ]:
# Indexing

In [ ]:
# Integration with Llama2